### **OPS 예측을 위한 크롤링**

*2018~2021년 4시즌간의 타자 날짜별 데이터를 수집* </br>
*팀당 30명씩 타자를 선정했으며 투수가 타석에 들어선 경우는 제외* 


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [39]:
import pandas as pd
import numpy as np

player_2018 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding='euc-kr')
player_2019 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding='euc-kr')
player_2020 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding='euc-kr')
player_2021 = pd.read_csv('/content/drive/MyDrive/bigcontest/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding='euc-kr')
player = pd.concat([player_2018,player_2019,player_2020,player_2021], ignore_index=True)

In [41]:
player_2021.loc[(player_2021['T_ID'] == 'HH'), 'T_ID'] = '한화'
player_2021.loc[(player_2021['T_ID'] == 'LT'), 'T_ID'] = '롯데'
player_2021.loc[(player_2021['T_ID'] == 'SS'), 'T_ID'] = '삼성'
player_2021.loc[(player_2021['T_ID'] == 'HT'), 'T_ID'] = 'KIA'
player_2021.loc[(player_2021['T_ID'] == 'SK'), 'T_ID'] = 'SSG'
player_2021.loc[(player_2021['T_ID'] == 'LG'), 'T_ID'] = 'LG'
player_2021.loc[(player_2021['T_ID'] == 'KT'), 'T_ID'] = 'kt'
player_2021.loc[(player_2021['T_ID'] == 'WO'), 'T_ID'] = '키움'
player_2021.loc[(player_2021['T_ID'] == 'OB'), 'T_ID'] = '두산'
player_2021.loc[(player_2021['T_ID'] == 'NC'), 'T_ID'] = 'NC'

In [42]:
player_2020.loc[(player_2020['T_ID'] == 'HH'), 'T_ID'] = '한화'
player_2020.loc[(player_2020['T_ID'] == 'LT'), 'T_ID'] = '롯데'
player_2020.loc[(player_2020['T_ID'] == 'SS'), 'T_ID'] = '삼성'
player_2020.loc[(player_2020['T_ID'] == 'HT'), 'T_ID'] = 'KIA'
player_2020.loc[(player_2020['T_ID'] == 'SK'), 'T_ID'] = 'SK'
player_2020.loc[(player_2020['T_ID'] == 'LG'), 'T_ID'] = 'LG'
player_2020.loc[(player_2020['T_ID'] == 'KT'), 'T_ID'] = 'kt'
player_2020.loc[(player_2020['T_ID'] == 'WO'), 'T_ID'] = '키움'
player_2020.loc[(player_2020['T_ID'] == 'OB'), 'T_ID'] = '두산'
player_2020.loc[(player_2020['T_ID'] == 'NC'), 'T_ID'] = 'NC'

In [43]:
player_2019.loc[(player_2019['T_ID'] == 'HH'), 'T_ID'] = '한화'
player_2019.loc[(player_2019['T_ID'] == 'LT'), 'T_ID'] = '롯데'
player_2019.loc[(player_2019['T_ID'] == 'SS'), 'T_ID'] = '삼성'
player_2019.loc[(player_2019['T_ID'] == 'HT'), 'T_ID'] = 'KIA'
player_2019.loc[(player_2019['T_ID'] == 'SK'), 'T_ID'] = 'SK'
player_2019.loc[(player_2019['T_ID'] == 'LG'), 'T_ID'] = 'LG'
player_2019.loc[(player_2019['T_ID'] == 'KT'), 'T_ID'] = 'kt'
player_2019.loc[(player_2019['T_ID'] == 'WO'), 'T_ID'] = '키움'
player_2019.loc[(player_2019['T_ID'] == 'OB'), 'T_ID'] = '두산'
player_2019.loc[(player_2019['T_ID'] == 'NC'), 'T_ID'] = 'NC'

In [44]:
player_2018.loc[(player_2018['T_ID'] == 'HH'), 'T_ID'] = '한화'
player_2018.loc[(player_2018['T_ID'] == 'LT'), 'T_ID'] = '롯데'
player_2018.loc[(player_2018['T_ID'] == 'SS'), 'T_ID'] = '삼성'
player_2018.loc[(player_2018['T_ID'] == 'HT'), 'T_ID'] = 'KIA'
player_2018.loc[(player_2018['T_ID'] == 'SK'), 'T_ID'] = 'SK'
player_2018.loc[(player_2018['T_ID'] == 'LG'), 'T_ID'] = 'LG'
player_2018.loc[(player_2018['T_ID'] == 'KT'), 'T_ID'] = 'kt'
player_2018.loc[(player_2018['T_ID'] == 'WO'), 'T_ID'] = '히어로즈'
player_2018.loc[(player_2018['T_ID'] == 'OB'), 'T_ID'] = '두산'
player_2018.loc[(player_2018['T_ID'] == 'NC'), 'T_ID'] = 'NC'

In [49]:
player_2021.rename(columns={'NAME':'name', 'T_ID':'Team'}, inplace=True)
player_2020.rename(columns={'NAME':'name', 'T_ID':'Team'}, inplace=True)
player_2019.rename(columns={'NAME':'name', 'T_ID':'Team'}, inplace=True)
player_2018.rename(columns={'NAME':'name', 'T_ID':'Team'}, inplace=True)

In [61]:
def data_crawling(year):
  # Parameter
  # 원하는 연도 입력
  year = year
  # 데이터를 수집할 타자 중 타석 수 상위 명수
  num_top_batter = 30

  # Team List
  # 원하는 연도에 등록된 팀을 미리 넣어두어야함 (추가 가능)
  Team_2016 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2017 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2018 = ['NC','kt','LG','두산','히어로즈','KIA','롯데','삼성','SK','한화']
  Team_2019 = ['NC','kt','LG','두산','키움','KIA','롯데','삼성','SK','한화']
  Team_2020 = ['NC','kt','LG','두산','키움','KIA','롯데','삼성','SK','한화']
  Team_2021 = ['kt','삼성','LG','SSG','NC','키움','두산','롯데','KIA','한화']

  # Team Dictionary
  # Team List를 딕셔너리에 넣음 (추가 가능)
  TeamDic = {'2016':Team_2016, '2017':Team_2017, '2018':Team_2018, '2019':Team_2019, '2020':Team_2020, '2021':Team_2021}

  ###############
  # BeautifulSoup

  import numpy as np
  import pandas as pd
  import requests
  from bs4 import BeautifulSoup
  from tqdm import tqdm

  # 데이터를 모을 dataframe 생성
  data = pd.DataFrame(columns=['name','Team', 'day', 'vs', 'result', 'bat_order', 'position', 'start_member', 'ab', 'score', 'hit', '2_hit', '3_hit', 'homerun', 'tb', 'rbi', 'sb', 'cs',
                              'bb', 'hpp', 'ibb', 'so', 'gdp', 'sh', 'sf', 'avg', 'obp', 'slg', 'ops', 'pitch', 'avLI', 'RE24', 'WPA'])
  batter_data = pd.DataFrame(columns=['Team','name', 'gyear', 'pcode'])

  for Team in tqdm(TeamDic[str(year)]): # 선택한 연도의 팀을 하나씩 가져옴
    # 스탯티즈 시즌기록실 페이지에서 연도, 팀 선택하고, 타석순으로 정렬
    wp = requests.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=" + str(year) + "&ye=" + str(year) + "&se=0&te=" + Team + "&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=TPA&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=30&si=&cn=")
    soup = BeautifulSoup(wp.text, "html.parser")
    
    # 현재 팀의 타석 수 상위 num_top_batter 명
    tmp_batter_list = np.empty(num_top_batter,dtype=object)
    tmp_team = np.array([Team]*num_top_batter) # 타석 수 상위 타자 팀명 저장
    tmp_batter_list_href = np.empty(num_top_batter,dtype=object)
    tmp_batter_ps=0
    
    for batter in soup.find_all('tr')[2:12]:
      tmp_batter_list[tmp_batter_ps] = batter.find_all('td')[1].text # 타석 수 상위 타자 이름 저장
      tmp_team[tmp_batter_ps] = Team
      tmp_batter_list_href[tmp_batter_ps] = batter.find_all('a')[0].attrs['href'] # 타석 수 상위 타자 주소 저장
      tmp_batter_ps+=1

    for batter in soup.find_all('tr')[14:24]:
      tmp_batter_list[tmp_batter_ps] = batter.find_all('td')[1].text # 타석 수 상위 타자 이름 저장
      tmp_team[tmp_batter_ps] = Team
      tmp_batter_list_href[tmp_batter_ps] = batter.find_all('a')[0].attrs['href'] # 타석 수 상위 타자 주소 저장
      tmp_batter_ps+=1

    for batter in soup.find_all('tr')[26:36]:
      if '|' in batter.find_all('td')[1].text:
        break;
      else:
        tmp_batter_list[tmp_batter_ps] = batter.find_all('td')[1].text # 타석 수 상위 타자 이름 저장
        tmp_team[tmp_batter_ps] = Team
        tmp_batter_list_href[tmp_batter_ps] = batter.find_all('a')[0].attrs['href'] # 타석 수 상위 타자 주소 저장
        tmp_batter_ps+=1

    batter_data = pd.concat([batter_data, pd.DataFrame({'Team':tmp_team, 'name':tmp_batter_list})])

    # 타석 수 상위 타자들 중 한명씩 데이터를 모으는 과정
    for top_batter in range(num_top_batter):
      if tmp_batter_list[top_batter] is None:
        continue

      wp=requests.get("http://www.statiz.co.kr/"+tmp_batter_list_href[top_batter]) # 타자 주소로 접근
      soup=BeautifulSoup(wp.text, "html.parser")

      # 날짜별 데이터로 접근
      href_byDate = soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-of-type(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table')[0].find_all('a')[3].attrs['href']
      wp=requests.get("http://www.statiz.co.kr/"+href_byDate)
      soup=BeautifulSoup(wp.text,"html.parser")

      # 타자/투수 : 타자로 접근
      href_by_batter=soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-of-type(2) > div > div:nth-of-type(1) > div:nth-of-type(1)')[0].find_all('a')[0].attrs['href']
      wp=requests.get("http://www.statiz.co.kr/"+href_by_batter)
      soup=BeautifulSoup(wp.text,"html.parser")
      
      # year : 원하는 연도로 접근
      try:
        href_byDate_year=soup.select('body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-of-type(2) > div > div:nth-of-type(2) > div')[0].find('a', text=str(year)).attrs['href']
        wp=requests.get("http://www.statiz.co.kr/"+href_byDate_year)
        soup=BeautifulSoup(wp.text,"html.parser")
      except:
        pass

      ## 데이터가 있는 페이지로 이동 완료
      ## 이제 실질적인 데이터를 모을 차례
      rawdata=soup.select('body > div > div.content-wrapper > div > section.content > div > div:nth-of-type(2) > div > div:nth-of-type(3) > div > div > table')[0]
      # rawdata : 전체 테이블

      # th를 제외한 순수한 경기 데이터 / th는 컬럼명 행이다.
      rawdata_not_th = rawdata.find_all('tr',{'class':['oddrow_stz0','evenrow_stz0']})

      # 행렬만들기
      rawdata_name = np.array([tmp_batter_list[top_batter]]*len(rawdata_not_th))
      rawdata_Team = np.array([tmp_team[top_batter]]*len(rawdata_not_th))
      rawdata_day = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_vs = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_result = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_bat_order = np.zeros(len(rawdata_not_th),dtype=np.int_)
      rawdata_position = np.zeros(len(rawdata_not_th), dtype=object)
      rawdata_start_memeber = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_ab = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_score = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_hit = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_2_hit = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_3_hit = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_homerun = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_tb = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_rbi = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_sb = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_cs = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_bb = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_hpp = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_ibb = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_so = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_gdp = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_sh = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rewdata_sf = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_avg = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_obp = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_slg = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_ops = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_pitch = np.zeros(len(rawdata_not_th), dtype=np.int_)
      rawdata_avLI = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_RE24 = np.zeros(len(rawdata_not_th), dtype=np.float)
      rawdata_WPA = np.zeros(len(rawdata_not_th), dtype=np.float)

      for rawdata_not_th_tr_index in range(len(rawdata_not_th)):
        # 한 행 : rawdata_not_th_tr_index
        rawdata_not_th_td = [text for text in rawdata_not_th[rawdata_not_th_tr_index].stripped_strings]
        # 현재 행의 하나의 값 : rawdata_not_th_td[index]
        # 데이터 입력
        rawdata_day[rawdata_not_th_tr_index] = rawdata_not_th_td[1]
        rawdata_vs[rawdata_not_th_tr_index] = rawdata_not_th_td[2]
        rawdata_result[rawdata_not_th_tr_index] = rawdata_not_th_td[3]+" "+rawdata_not_th_td[4]
        rawdata_bat_order[rawdata_not_th_tr_index] = rawdata_not_th_td[5]
        rawdata_position[rawdata_not_th_tr_index] = rawdata_not_th_td[6]
        rawdata_start_memeber[rawdata_not_th_tr_index] = rawdata_not_th_td[7]
        rawdata_ab[rawdata_not_th_tr_index] = rawdata_not_th_td[8]
        rawdata_score[rawdata_not_th_tr_index] = rawdata_not_th_td[9]
        rawdata_hit[rawdata_not_th_tr_index] = rawdata_not_th_td[10]
        rawdata_2_hit[rawdata_not_th_tr_index] = rawdata_not_th_td[11]
        rawdata_3_hit[rawdata_not_th_tr_index] = rawdata_not_th_td[12]
        rawdata_homerun[rawdata_not_th_tr_index] = rawdata_not_th_td[13]
        rawdata_tb[rawdata_not_th_tr_index] = rawdata_not_th_td[14]
        rawdata_rbi[rawdata_not_th_tr_index] = rawdata_not_th_td[15]
        rawdata_sb[rawdata_not_th_tr_index] = rawdata_not_th_td[16]
        rawdata_cs[rawdata_not_th_tr_index] = rawdata_not_th_td[17]
        rawdata_bb[rawdata_not_th_tr_index] = rawdata_not_th_td[18]
        rawdata_hpp[rawdata_not_th_tr_index] = rawdata_not_th_td[19]
        rawdata_ibb[rawdata_not_th_tr_index] = rawdata_not_th_td[20]
        rawdata_so[rawdata_not_th_tr_index] = rawdata_not_th_td[21]
        rawdata_gdp[rawdata_not_th_tr_index] = rawdata_not_th_td[22]
        rawdata_sh[rawdata_not_th_tr_index] = rawdata_not_th_td[23]
        rewdata_sf[rawdata_not_th_tr_index] = rawdata_not_th_td[24]
        rawdata_avg[rawdata_not_th_tr_index] = rawdata_not_th_td[25]
        rawdata_obp[rawdata_not_th_tr_index] = rawdata_not_th_td[26]
        rawdata_slg[rawdata_not_th_tr_index] = rawdata_not_th_td[27]
        rawdata_ops[rawdata_not_th_tr_index] = rawdata_not_th_td[28]
        rawdata_pitch[rawdata_not_th_tr_index] = rawdata_not_th_td[29]
        rawdata_avLI[rawdata_not_th_tr_index] = rawdata_not_th_td[30]
        rawdata_RE24[rawdata_not_th_tr_index] = rawdata_not_th_td[31]
        rawdata_WPA[rawdata_not_th_tr_index] = rawdata_not_th_td[32]

      tmp_data = pd.DataFrame()
      # tmp_data는 각 선수마다 데이터를 모은 후 data에 머지하기 전에 존재
      tmp_data['name'] = rawdata_name
      tmp_data['Team'] = rawdata_Team
      tmp_data['day'] = rawdata_day
      tmp_data['vs'] = rawdata_vs
      tmp_data['result'] = rawdata_result
      tmp_data['bat_order'] = rawdata_bat_order
      tmp_data['position'] = rawdata_position
      tmp_data['start_member'] = rawdata_start_memeber
      tmp_data['ab'] = rawdata_ab
      tmp_data['score'] = rawdata_score
      tmp_data['hit'] = rawdata_hit
      tmp_data['2_hit'] = rawdata_2_hit
      tmp_data['3_hit'] = rawdata_3_hit
      tmp_data['homerun'] = rawdata_homerun
      tmp_data['tb'] = rawdata_tb
      tmp_data['rbi'] = rawdata_rbi
      tmp_data['sb'] = rawdata_sb
      tmp_data['cs'] = rawdata_cs
      tmp_data['bb'] = rawdata_bb
      tmp_data['hpp'] = rawdata_hpp
      tmp_data['ibb'] = rawdata_ibb
      tmp_data['so'] = rawdata_so
      tmp_data['gdp'] = rawdata_gdp
      tmp_data['sh'] = rawdata_sh
      tmp_data['sf'] = rewdata_sf
      tmp_data['avg'] = rawdata_avg
      tmp_data['ops'] = rawdata_ops
      tmp_data['slg'] = rawdata_slg
      tmp_data['ops'] = rawdata_ops
      tmp_data['pitch'] = rawdata_pitch
      tmp_data['avLI'] = rawdata_avLI
      tmp_data['RE24'] = rawdata_RE24
      tmp_data['WPA'] = rawdata_WPA

      # 데이터 쌓기
      data = pd.concat([data,tmp_data])

  return data, batter_data

### **2021 season**

In [ ]:
ops_data_2021, ops_batter_data_2021 = data_crawling(2021)

In [ ]:
name_list_2021 = ops_data_2021['name'].values.tolist()
name_set_2021 = set(name_list_2021)
name_list_2021 = list(name_set_2021)

In [ ]:
len(name_list_2021)

In [ ]:
new_ops_data_2021 = ops_data_2021.merge(player_2021, on=['Team','name'])

In [ ]:
name_list_2021_2 = new_ops_data_2021['name'].values.tolist()
name_set_2021_2 = set(name_list_2021_2)
name_list_2021_2 = list(name_set_2021_2)

In [ ]:
diff_2021 = list(set(name_list_2021) - set(name_list_2021_2))
diff_2021

In [ ]:
player_2021.loc[player_2021['name']=='한동민']

In [ ]:
ops_data_2021.loc[ops_data_2021['name']=='한유섬']

In [ ]:
#임동휘, 윤여운, 백진우, 한유섬
#개명하여 PCODE를 불러올 수 없음
#따라서 PCODE가 저장된 선수 파일에서 개명한 이름을 최신화
player_2021.loc[(player_2021['name'] == '임지열'), 'name'] = '임동휘'
player_2021.loc[(player_2021['name'] == '윤수강'), 'name'] = '윤여운'
player_2021.loc[(player_2021['name'] == '백창수'), 'name'] = '백진우'
player_2021.loc[(player_2021['name'] == '한동민'), 'name'] = '한유섬'

In [ ]:
final_ops_data_2021 = ops_data_2021.merge(player_2021, on=['Team','name'])

In [ ]:
final_ops_data_2021['name'].value_counts()

In [ ]:
final_ops_data_2021.to_csv("final_ops_data_2021.csv", encoding='euc-kr', index=False)
ops_batter_data_2021.to_csv("final_ops_batter_data_2021.csv", encoding='euc-kr', index=False)

### **2020 season**

In [159]:
ops_data_2020, ops_batter_data_2020 = data_crawling(2020)

100%|██████████| 10/10 [51:40<00:00, 310.03s/it]


In [160]:
name_list_2020 = ops_data_2020['name'].values.tolist()
name_set_2020 = set(name_list_2020)
name_list_2020 = list(name_set_2020)

In [161]:
len(name_list_2020)

288

In [162]:
new_ops_data_2020 = ops_data_2020.merge(player_2020, on=['Team','name'])

In [163]:
name_list_2020_2 = new_ops_data_2020['name'].values.tolist()
name_set_2020_2 = set(name_list_2020_2)
name_list_2020_2 = list(name_set_2020_2)

In [164]:
diff_2020 = list(set(name_list_2020) - set(name_list_2020_2))
diff_2020

['장현식', '한유섬']

In [166]:
player_2020.loc[player_2020['name']=='장현식']

,GYEAR,PCODE,name,Team,POSITION,AGE_VA,MONEY
213,2020,63950,장현식,KIA,투,25,11000만원


In [167]:
ops_data_2020.loc[ops_data_2020['name']=='장현식']

,name,Team,day,vs,result,bat_order,position,start_member,ab,score,hit,2_hit,3_hit,homerun,tb,rbi,sb,cs,bb,hpp,ibb,so,gdp,sh,sf,avg,obp,slg,ops,pitch,avLI,RE24,WPA
0,장현식,NC,05-21,@두산,W 12:6,5,P,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,NaN,0.0,0.0,0,0.0,0.0,0.0


In [168]:
#한유섬
#개명하여 PCODE를 불러올 수 없음
#따라서 PCODE가 저장된 선수 파일에서 개명한 이름을 최신화
player_2020.loc[(player_2020['name'] == '한동민'), 'name'] = '한유섬'

In [169]:
final_ops_data_2020 = ops_data_2020.merge(player_2020, on=['Team','name'])

In [170]:
final_ops_data_2020['name'].value_counts()

페르난데스    156
정수빈      153
김재환      152
최주환      150
심우준      148
        ... 
이호연        1
라이블리       1
조병욱        1
권휘         1
김대우        1
Name: name, Length: 287, dtype: int64

In [171]:
final_ops_data_2020.to_csv("final_ops_data_2020.csv", encoding='euc-kr', index=False)
ops_batter_data_2020.to_csv("final_ops_batter_data_2020.csv", encoding='euc-kr', index=False)

### **2019 season**

In [131]:
ops_data_2019, ops_batter_data_2019 = data_crawling(2019)

100%|██████████| 10/10 [41:25<00:00, 248.52s/it]


In [132]:
name_list_2019 = ops_data_2019['name'].values.tolist()
name_set_2019 = set(name_list_2019)
name_list_2019 = list(name_set_2019)

In [133]:
len(name_list_2019)

284

In [134]:
new_ops_data_2019 = ops_data_2019.merge(player_2019, on=['Team','name'])

In [135]:
name_list_2019_2 = new_ops_data_2019['name'].values.tolist()
name_set_2019_2 = set(name_list_2019_2)
name_list_2019_2 = list(name_set_2019_2)

In [136]:
diff_2019 = list(set(name_list_2019) - set(name_list_2019_2))
diff_2019

['임동휘', '신정락', '오승택', '윤여운', '백진우', '한유섬']

In [152]:
player_2019.loc[player_2019['name']=='한동민']

,GYEAR,PCODE,name,Team,POSITION,AGE_VA,MONEY
97,2019,62895,한동민,SK,외,29,33000만원


In [ ]:
ops_data_2019.loc[ops_data_2019['name']=='한유섬']

In [155]:
#임동휘, 윤여운, 백진우, 한유섬
#개명하여 PCODE를 불러올 수 없음
#따라서 PCODE가 저장된 선수 파일에서 개명한 이름을 최신화
player_2019.loc[(player_2019['name'] == '임지열'), 'name'] = '임동휘'
player_2019.loc[(player_2019['name'] == '윤수강'), 'name'] = '윤여운'
player_2019.loc[(player_2019['name'] == '백창수'), 'name'] = '백진우'
player_2019.loc[(player_2019['name'] == '한동민'), 'name'] = '한유섬'

In [156]:
final_ops_data_2019 = ops_data_2019.merge(player_2019, on=['Team','name'])

In [157]:
final_ops_data_2019['name'].value_counts()

이정후      151
샌즈       150
김하성      150
페르난데스    148
김성현      147
        ... 
이영하        1
양우현        1
양원혁        1
권혁         1
추재현        1
Name: name, Length: 282, dtype: int64

In [158]:
final_ops_data_2019.to_csv("final_ops_data_2019.csv", encoding='euc-kr', index=False)
ops_batter_data_2019.to_csv("final_ops_batter_data_2019.csv", encoding='euc-kr', index=False)

### **2018 season**

In [76]:
ops_data_2018, ops_batter_data_2018 = data_crawling(2018)

100%|██████████| 10/10 [42:07<00:00, 252.80s/it]


In [104]:
name_list_2018 = ops_data_2018['name'].values.tolist()
name_set_2018 = set(name_list_2018)
name_list_2018 = list(name_set_2018)

In [105]:
new_ops_data_2018 = ops_data_2018.merge(player_2018, on=['Team','name'])

In [106]:
name_list_2018_2 = new_ops_data_2018['name'].values.tolist()
name_set_2018_2 = set(name_list_2018_2)
name_list_2018_2 = list(name_set_2018_2)

In [107]:
diff_2018 = list(set(name_list_2018) - set(name_list_2018_2))
diff_2018

['백동훈', '박대온', '윤여운', '백진우', '윤형준', '한유섬', '이유찬']

In [123]:
player_2018.loc[player_2018['name']=='백진우']

,GYEAR,PCODE,name,Team,POSITION,AGE_VA,MONEY
0,2018,60100,백진우,한화,외,30,5000만원


In [ ]:
ops_data_2018.loc[ops_data_2018['name']=='백진우']

In [121]:
#백동훈, 박대온, 윤여운, 백진우, 윤형준, 한유섬, 이유찬
#개명하여 PCODE를 불러올 수 없음
#따라서 PCODE가 저장된 선수 파일에서 개명한 이름을 최신화
player_2018.loc[(player_2018['name'] == '백민기'), 'name'] = '백동훈'
player_2018.loc[(player_2018['name'] == '박광열'), 'name'] = '박대온'
player_2018.loc[(player_2018['name'] == '윤수강'), 'name'] = '윤여운'
player_2018.loc[(player_2018['name'] == '백창수'), 'name'] = '백진우'
player_2018.loc[(player_2018['name'] == '윤대영'), 'name'] = '윤형준'
player_2018.loc[(player_2018['name'] == '한동민'), 'name'] = '한유섬'
player_2018.loc[(player_2018['name'] == '이병휘'), 'name'] = '이유찬'

In [124]:
final_ops_data_2018 = ops_data_2018.merge(player_2018, on=['Team','name'])

In [127]:
final_ops_data_2018['name'].value_counts()

김재현    191
로맥     152
한유섬    147
호잉     146
김성현    146
      ... 
구승민      1
이윤재      1
김성민      1
손승락      1
송창식      1
Name: name, Length: 285, dtype: int64

In [130]:
final_ops_data_2018.to_csv("final_ops_data_2018.csv", encoding='euc-kr', index=False)
ops_batter_data_2018.to_csv("final_ops_batter_data_2018.csv", encoding='euc-kr', index=False)

### **2017 season**

In [ ]:
ops_data_2017, ops_batter_data_2017 = data_crawling(2017)

In [ ]:
name_list_2017 = ops_data_2017['name'].values.tolist()
name_set_2017 = set(name_list_2017)
name_list_2017 = list(name_set_2017)

In [ ]:
len(name_list_2017)

In [ ]:
new_ops_data_2017 = ops_data_2017.merge(player_2018, on=['Team','name'])

In [ ]:
name_list_2017_2 = new_ops_data_2017['name'].values.tolist()
name_set_2017_2 = set(name_list_2017_2)
name_list_2017_2 = list(name_set_2017_2)

In [ ]:
diff_2017 = list(set(name_list_2017) - set(name_list_2017_2))
diff_2017

In [ ]:
player_2018.loc[player_2018['name']=='백진우']

In [ ]:
ops_data_2017.loc[ops_data_2017['name']=='백진우']

In [ ]:
#백동훈, 박대온, 윤여운, 백진우, 윤형준, 한유섬, 이유찬
#개명하여 PCODE를 불러올 수 없음
#따라서 PCODE가 저장된 선수 파일에서 개명한 이름을 최신화
player_2018.loc[(player_2018['name'] == '백민기'), 'name'] = '백동훈'
player_2018.loc[(player_2018['name'] == '박광열'), 'name'] = '박대온'
player_2018.loc[(player_2018['name'] == '윤수강'), 'name'] = '윤여운'
player_2018.loc[(player_2018['name'] == '백창수'), 'name'] = '백진우'
player_2018.loc[(player_2018['name'] == '윤대영'), 'name'] = '윤형준'
player_2018.loc[(player_2018['name'] == '한동민'), 'name'] = '한유섬'
player_2018.loc[(player_2018['name'] == '이병휘'), 'name'] = '이유찬'

In [ ]:
final_ops_data_2017 = ops_data_2017.merge(player_2018, on=['Team','name'])

In [ ]:
final_ops_data_2017['name'].value_counts()

In [ ]:
final_ops_data_2017.to_csv("final_ops_data_2017.csv", encoding='euc-kr', index=False)
ops_batter_data_2017.to_csv("final_ops_batter_data_2017.csv", encoding='euc-kr', index=False)

### **2016 season**

In [ ]:
ops_data_2016, ops_batter_data_2016 = data_crawling(2016)

In [ ]:
name_list_2016 = ops_data_2016['name'].values.tolist()
name_set_2016 = set(name_list_2016)
name_list_2016 = list(name_set_2016)

In [ ]:
len(name_list_2016)

In [ ]:
new_ops_data_2016 = ops_data_2016.merge(player_2018, on=['Team','name'])

In [ ]:
name_list_2016_2 = new_ops_data_2016['name'].values.tolist()
name_set_2016_2 = set(name_list_2016_2)
name_list_2016_2 = list(name_set_2016_2)

In [ ]:
diff_2016 = list(set(name_list_2016) - set(name_list_2016_2))
diff_2016

In [ ]:
player_2018.loc[player_2018['name']=='백진우']

In [ ]:
ops_data_2016.loc[ops_data_2016['name']=='백진우']

In [ ]:
#백동훈, 박대온, 윤여운, 백진우, 윤형준, 한유섬, 이유찬
#개명하여 PCODE를 불러올 수 없음
#따라서 PCODE가 저장된 선수 파일에서 개명한 이름을 최신화
player_2018.loc[(player_2018['name'] == '백민기'), 'name'] = '백동훈'
player_2018.loc[(player_2018['name'] == '박광열'), 'name'] = '박대온'
player_2018.loc[(player_2018['name'] == '윤수강'), 'name'] = '윤여운'
player_2018.loc[(player_2018['name'] == '백창수'), 'name'] = '백진우'
player_2018.loc[(player_2018['name'] == '윤대영'), 'name'] = '윤형준'
player_2018.loc[(player_2018['name'] == '한동민'), 'name'] = '한유섬'
player_2018.loc[(player_2018['name'] == '이병휘'), 'name'] = '이유찬'

In [ ]:
final_ops_data_2016 = ops_data_2016.merge(player_2018, on=['Team','name'])

In [ ]:
final_ops_data_2016['name'].value_counts()

In [ ]:
final_ops_data_2016.to_csv("final_ops_data_2016.csv", encoding='euc-kr', index=False)
ops_batter_data_2016.to_csv("final_ops_batter_data_2016.csv", encoding='euc-kr', index=False)